In [ ]:
pip install --upgrade opencv-contrib-python== 4.10.0.84


In [ ]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision


In [ ]:
pip install mediapipe

In [ ]:
import tensorflow as tf
print(tf.__version__)


In [22]:
import cv2
import mediapipe as mp

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

with mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break


        frame = cv2.flip(frame, 1)
  
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        results = hands.process(rgb_frame)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                handedness = results.multi_handedness[0].classification[0].label
                if handedness == 'Right':
  
                    mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                    all_fingers_right = True
                    all_fingers_left = True
                    all_fingers_up = True
                    all_fingers_down = True

                    for id in range(5): 
      
                        tip = hand_landmarks.landmark[mp_hands.HandLandmark(id * 4 + 4)]
                        pip = hand_landmarks.landmark[mp_hands.HandLandmark(id * 4 + 2)]
                        mcp = hand_landmarks.landmark[mp_hands.HandLandmark(id * 4 + 1)]
                        
                        if tip.x < pip.x:
                            all_fingers_right = False

                        if tip.x > pip.x:
                            all_fingers_left = False

                        if tip.y > pip.y:
                            all_fingers_up = False
                        
                        if tip.y < mcp.y:
                            all_fingers_down = False

                    if all_fingers_right:
                        gesture = "Right"
                    elif all_fingers_left:
                        gesture = "Left"
                    elif all_fingers_up:
                        gesture = "Break"
                    elif all_fingers_down:
                        gesture = "MoveBack"
                    else:
                        gesture = "go"

                    if gesture:
                        cv2.putText(frame, gesture, (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        cv2.imshow('Hand Gesture Recognition', frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import mediapipe as mp


mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils


cap = cv2.VideoCapture(0)


def is_finger_raised(hand_landmarks, finger_indices, finger_name, is_right_hand=True):
    if finger_name == 'thumb':
        if is_right_hand:
            return hand_landmarks.landmark[finger_indices[1]].x > hand_landmarks.landmark[finger_indices[0]].x
        else:
            return hand_landmarks.landmark[finger_indices[1]].x < hand_landmarks.landmark[finger_indices[0]].x
    else:
        return hand_landmarks.landmark[finger_indices[1]].y < hand_landmarks.landmark[finger_indices[0]].y


with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)

        # Convert the image to RGB
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        results = hands.process(image_rgb)

        output = 'other'
        if results.multi_hand_landmarks:
            for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
                hand_label = handedness.classification[0].label
                is_right_hand = hand_label == 'Right'

                finger_indices = {
                    'thumb': [mp_hands.HandLandmark.THUMB_MCP, mp_hands.HandLandmark.THUMB_TIP],
                    'index': [mp_hands.HandLandmark.INDEX_FINGER_MCP, mp_hands.HandLandmark.INDEX_FINGER_TIP],
                    'middle': [mp_hands.HandLandmark.MIDDLE_FINGER_MCP, mp_hands.HandLandmark.MIDDLE_FINGER_TIP],
                    'ring': [mp_hands.HandLandmark.RING_FINGER_MCP, mp_hands.HandLandmark.RING_FINGER_TIP],
                    'pinky': [mp_hands.HandLandmark.PINKY_MCP, mp_hands.HandLandmark.PINKY_TIP],
                }

                fingers_raised = {
                    'thumb': is_finger_raised(hand_landmarks, finger_indices['thumb'], 'thumb', is_right_hand),
                    'index': is_finger_raised(hand_landmarks, finger_indices['index'], 'index', is_right_hand),
                    'middle': is_finger_raised(hand_landmarks, finger_indices['middle'], 'middle', is_right_hand),
                    'ring': is_finger_raised(hand_landmarks, finger_indices['ring'], 'ring', is_right_hand),
                    'pinky': is_finger_raised(hand_landmarks, finger_indices['pinky'], 'pinky', is_right_hand),
                }
                print(fingers_raised)

                # Gesture detection logic (same as before)
                if fingers_raised['index'] and not fingers_raised['middle'] and not fingers_raised['ring']and not fingers_raised['pinky']and not fingers_raised['thumb']:
                    output = 1
                elif fingers_raised['index'] and fingers_raised['middle'] and not fingers_raised['ring']and not fingers_raised['pinky']and not fingers_raised['thumb']:
                    output = 2
                elif fingers_raised['index'] and fingers_raised['middle'] and fingers_raised['ring']and not fingers_raised['pinky']and not fingers_raised['thumb']:
                    output = 3
                else:
                    output = 'other'

                mp_drawing.draw_landmarks(
                    frame, 
                    hand_landmarks, 
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2)
                )

        cv2.putText(frame, f'Gesture: {output}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.imshow("Hand Tracking", frame)

        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 